#スクレイピング

In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2
from psycopg2 import sql

# スクレイピング関数
def scrape_transit_data(start_station, end_station):
    url = f"https://transit.yahoo.co.jp/search/result?from={start_station}&to={end_station}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    routes = []
    for route in soup.select('.routeDetail'):
        try:
            # 出発時刻と到着時刻
            times = route.select_one('.time').get_text().split('→')
            departure_time = times[0].strip()
            arrival_time = times[1].strip()
            
            # 乗り換え回数
            transfer_count = int(route.select_one('.transfer').get_text().replace('乗り換え', '').replace('回', ''))
            
            # 所要時間
            travel_time = route.select_one('.time').get_text().strip().split('(')[1].replace(')', '').replace('分', '')
            travel_time = int(travel_time)
            
            # 運賃
            fare = route.select_one('.fare').get_text().replace('¥', '').replace(',', '')
            fare = int(fare)
            
            # データをリストに追加
            routes.append({
                'departure_time': departure_time,
                'arrival_time': arrival_time,
                'transfer_count': transfer_count,
                'travel_time': travel_time,
                'fare': fare
            })
        except Exception as e:
            print(f"Error parsing route: {e}")
            continue

    return routes
